### 경기도내 인구소멸 위기지역 Top 10

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 1) 파일 읽기
pop = pd.read_csv('data/경기도_주민등록인구집계현황.csv', encoding='euc-kr')
pop.head()

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2022,12,도,경기도,13589432,1040954,1327726,1737291,1882300,2284603,...,647283,821150,904073,1121889,1133428,878494,437251,252711,44184,1369
1,2022,12,시군,경기도 가평군,62197,2930,4463,5929,5397,7589,...,2186,2503,2433,3525,5090,6439,3653,2408,411,13
2,2022,12,읍면동,경기도 가평군 가평읍,19555,1020,1673,2094,1795,2630,...,778,937,848,1244,1583,1805,1054,739,126,6
3,2022,12,읍면동,경기도 가평군 북면,3815,108,174,230,223,357,...,93,87,99,154,290,511,294,216,39,1
4,2022,12,읍면동,경기도 가평군 상면,5669,140,259,482,389,533,...,112,151,166,240,515,737,389,244,39,0


In [3]:
# 2) 결측치 확인
pop.isna().sum().sum()

0

In [4]:
# 3) 컬럼 선택
pop = pop[['행정구역구분명','행정구역명','60~69세','70~79세','80~89세','90~99세','100세 이상','20~29세 (여)','30~39세 (여)']]
pop.head(3)

,행정구역구분명,행정구역명,60~69세,70~79세,80~89세,90~99세,100세 이상,20~29세 (여),30~39세 (여)
0,도,경기도,1750962,808663,403721,58065,1725,821150,904073
1,시군,경기도 가평군,13315,7082,3999,564,19,2503,2433
2,읍면동,경기도 가평군 가평읍,3614,1963,1206,163,7,937,848


In [5]:
# 4) 원하는 컬럼 만들고, 필요한 컬럼만 선택
pop['인구소멸비율'] = (pop.iloc[:, -2:].sum(axis=1) / pop.iloc[:, 2:7].sum(axis=1)).round(2)
pop = pop[['행정구역구분명','행정구역명','인구소멸비율']]
pop.head()

,행정구역구분명,행정구역명,인구소멸비율
0,도,경기도,0.57
1,시군,경기도 가평군,0.20
2,읍면동,경기도 가평군 가평읍,0.26
3,읍면동,경기도 가평군 북면,0.09
4,읍면동,경기도 가평군 상면,0.11


In [6]:
# 특례시, 구(행정구)가 있는 도시
pop[pop.행정구역명.str.contains('안양')].head()

,행정구역구분명,행정구역명,인구소멸비율
344,시군,경기도 안양시,0.56
345,구,경기도 안양시 동안구,0.63
346,읍면동,경기도 안양시 동안구 갈산동,0.44
347,읍면동,경기도 안양시 동안구 관양1동,0.46
348,읍면동,경기도 안양시 동안구 관양2동,0.58


In [7]:
# 5) 원하는 행만 가져오기(Filtering)
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()

,행정구역구분명,행정구역명,인구소멸비율
1,시군,경기도 가평군,0.20
8,시군,경기도 고양시,0.57
9,구,경기도 고양시 덕양구,0.56
31,구,경기도 고양시 일산동구,0.60
44,구,경기도 고양시 일산서구,0.56


In [13]:
# 수원시, 용인시 등 구가 있는 시는 제거
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
drop_index

[8, 176, 231, 300, 344, 435]

In [14]:
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,인구소멸비율
1,시군,경기도 가평군,0.20
9,구,경기도 고양시 덕양구,0.56
31,구,경기도 고양시 일산동구,0.60
44,구,경기도 고양시 일산서구,0.56
56,시군,경기도 과천시,0.62


In [15]:
# 6) 인구소멸비율이 낮은 인구소멸위험지역 Top 10
pop.sort_values(by='인구소멸비율').head(10)

,행정구역구분명,행정구역명,인구소멸비율
1,시군,경기도 가평군,0.20
417,시군,경기도 연천군,0.20
391,시군,경기도 양평군,0.20
404,시군,경기도 여주시,0.27
558,시군,경기도 포천시,0.28
156,시군,경기도 동두천시,0.33
328,시군,경기도 안성시,0.38
464,구,경기도 용인시 처인구,0.48
363,구,경기도 안양시 만안구,0.48
218,구,경기도 성남시 중원구,0.50


In [16]:
# 인구소멸 비위험지역 Top 10
pop.sort_values(by='인구소멸비율', ascending=False).head(10)

,행정구역구분명,행정구역명,인구소멸비율
589,시군,경기도 화성시,0.87
428,시군,경기도 오산시,0.76
280,시군,경기도 시흥시,0.72
574,시군,경기도 하남시,0.67
452,구,경기도 용인시 수지구,0.66
177,구,경기도 성남시 분당구,0.66
532,시군,경기도 평택시,0.65
301,구,경기도 안산시 단원구,0.64
345,구,경기도 안양시 동안구,0.63
232,구,경기도 수원시 권선구,0.63
